 1. In the above dataset,
 a. In case age is less than 18, replace it with mean of age values.
 b. In case spend amount is more than the limit, replace it with 50% of that customer’s limit. 
(customer’s limit provided in acquisition table is the per transaction limit on his card)
 c. Incase the repayment amount is more than the limit, replace the repayment with the 
limit

In [ ]:
import pandas as pd

# Load the Customer_Final dataset
Customer_Final = pd.read_csv('path/to/Customer_Final.csv')

# Load the Acquisition table
Acquisition = pd.read_csv('path/to/Acquisition.csv')

In [ ]:
# Calculate the mean age value
mean_age = Customer_Final['Age'].mean()

# Replace age values less than 18 with the mean age value
Customer_Final.loc[Customer_Final['Age'] < 18, 'Age'] = mean_age

In [ ]:
# Merge the Customer_Final dataset with the Acquisition table
merged_data = pd.merge(Customer_Final, Acquisition[['Customer_ID', 'Limit']], on='Customer_ID', how='left')

# Replace spend amounts more than the limit with 50% of that customer's limit
merged_data['Transaction_Value'] = merged_data.apply(lambda row: min(row['Transaction_Value'], 0.5 * row['Limit']) if pd.notna(row['Limit']) else row['Transaction_Value'], axis=1)

 2. From the above dataset create the following summaries:
 a. How many distinct customers exist?
 b. How many distinct categories exist?
 c. What is the average monthly spend by customers?
 d. What is the average monthly repayment by customers?
 e. If the monthly rate of interest is 2.9%, what is the profit for the bank for each month? 
(Profit is defined as interest earned on Monthly Profit. Monthly Profit = Monthly repayment – Monthly spend. Interest is earned only on positive profits and not on negative amounts)
f. What are the top 5 product types?
 g. Which city is having maximum spend?
 h. Which age group is spending more money?
 i. Who are the top 10 customers in terms of repayment?

In [ ]:
import pandas as pd

# Load the Customer_Final dataset
Customer_Final = pd.read_csv('path/to/Customer_Final.csv')

In [ ]:
# Calculate the number of distinct customers
num_distinct_customers = Customer_Final['Customer_ID'].nunique()

# Calculate the number of distinct categories
num_distinct_categories = Customer_Final['Product_Category'].nunique()

# Calculate the average monthly spend by customers
avg_monthly_spend = Customer_Final.groupby(Customer_Final['Customer_ID'].str[:7])['Transaction_Value'].sum().mean()

# Calculate the average monthly repayment by customers
avg_monthly_repayment = Customer_Final.groupby(Customer_Final['Customer_ID'].str[:7])['Repayment_Amount'].sum().mean()

# Calculate the monthly profit for the bank
monthly_profit = avg_monthly_repayment - avg_monthly_spend
monthly_interest = monthly_profit * 0.029
bank_profit = monthly_interest[monthly_interest > 0].sum()

# Calculate the top 5 product types
top_5_product_types = Customer_Final['Product_Category'].value_counts().head(5)

# Calculate the city with maximum spend
max_spend_city = Customer_Final.groupby('City')['Transaction_Value'].sum().idxmax()

# Calculate the age group with maximum spend
age_groups = [f'Age_{i}' for i in range(18, 101, 5)]
age_group_spend = Customer_Final.groupby(Customer_Final['Age'].astype(str).str[:3])['Transaction_Value'].sum()
max_spend_age_group = age_group_spend.idxmax()

# Calculate the top 10 customers in terms of repayment
top_10_customers = Customer_Final.groupby('Customer_ID')['Repayment_Amount'].sum().sort_values(ascending=False).head(10)

In [ ]:
print(f"a. Number of distinct customers: {num_distinct_customers}")
print(f"b. Number of distinct categories: {num_distinct_categories}")
print(f"c. Average monthly spend by customers: ${avg_monthly_spend:,.2f}")
print(f"d. Average monthly repayment by customers: ${avg_monthly_repayment:,.2f}")
print(f"e. Bank profit for each month: ${bank_profit:,.2f}")
print("f. Top 5 product types:")
print(top_5_product_types)
print(f"g. City with maximum spend: {max_spend_city}")
print(f"h. Age group with maximum spend: {max_spend_age_group}")
print("i. Top 10 customers in terms of repayment:")
print(top_10_customers)

 3. Calculate the city wise spend on each product on yearly basis. Also include a graphical 
representation for the same.

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Load the merged dataset
merged_data = pd.read_csv('path/to/merged_dataset.csv')

# Create a new DataFrame for yearly spend
yearly_spend = pd.DataFrame(columns=['City', 'Product', 'Year', 'Yearly_Spend'])

# Extract year, city, product, and transaction value
merged_data['Transaction_Date'] = pd.to_datetime(merged_data['Transaction_Date'])
merged_data['Year'] = merged_data['Transaction_Date'].dt.year

# Calculate yearly spend for each city and product
for index, row in merged_data.iterrows():
    yearly_spend_row = {
        'City': row['City'],
        'Product': row['Product'],
        'Year': row['Year'],
        'Yearly_Spend': row['Transaction_Value']
    }
    yearly_spend = yearly_spend.append(yearly_spend_row, ignore_index=True)

# Group the data by city, product, and yearly spend
yearly_spend_grouped = yearly_spend.groupby(['City', 'Product', 'Year']).sum().reset_index()

# Create a heatmap or bar chart
plt.figure(figsize=(16, 12))
heatmap_data = yearly_spend_grouped.pivot(index='City', columns='Product', values='Yearly_Spend')
sns.heatmap(heatmap_data, annot=True, cmap='YlGnBu')
plt.title('City-wise Yearly Spend on Each Product')
plt.show()

 4. Create graphs for
 a. Monthly comparison of total spends, city wise
 b. Comparison of yearly spend on air tickets
 c. Comparison of monthly spend for each product (look for any seasonality
 that exists in terms of spend)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the merged dataset
merged_data = pd.read_csv('path/to/merged_dataset.csv')

# Filter the dataset for each city
city_data = merged_data.groupby('City').filter(lambda x: len(x) > 0)

# Calculate the total spend for each month in each city
monthly_spend = city_data.groupby(['City', 'Transaction_Date.month'])['Transaction_Value'].sum().reset_index()

# Pivot the data to create a multi-index DataFrame
monthly_spend_pivot = monthly_spend.pivot(index='City', columns='Transaction_Date.month', values='Transaction_Value')

# Plot the data as a line graph
plt.figure(figsize=(12, 6))
monthly_spend_pivot.plot(kind='line')
plt.title('Monthly Comparison of Total Spends, City Wise')
plt.xlabel('City')
plt.ylabel('Total Spend')
plt.show()

In [ ]:
# Filter the merged dataset for air ticket transactions only
air_tickets = merged_data[merged_data['Product'] == 'Air Tickets']

# Calculate the total spend for each year on air tickets
yearly_spend = air_tickets.groupby('Transaction_Date.year')['Transaction_Value'].sum().reset_index()

# Plot the data as a bar graph
plt.figure(figsize=(8, 6))
plt.bar(yearly_spend['Transaction_Date.year'], yearly_spend['Transaction_Value'])
plt.title('Comparison of Yearly Spend on Air Tickets')
plt.xlabel('Year')
plt.ylabel('Total Spend')
plt.show()

In [ ]:
# Calculate the total spend for each month for each product
monthly_product_spend = merged_data.groupby(['Product', 'Transaction_Date.month'])['Transaction_Value'].sum().reset_index()

# Pivot the data to create a multi-index DataFrame
monthly_product_spend_pivot = monthly_product_spend.pivot(index='Product', columns='Transaction_Date.month', values='Transaction_Value')

# Plot the data as a line graph
plt.figure(figsize=(12, 6))
monthly_product_spend_pivot.plot(kind='line')
plt.title('Comparison of Monthly Spend for Each Product')
plt.xlabel('Month')
plt.ylabel('Total Spend')
plt.show()

 5. Write user defined PYTHON function to perform the following analysis:
 You need to find top 10 customers for each city in terms of their repayment amount by 
different products and by different time periods i.e. year or month. The user should be able 
to specify the product (Gold/Silver/Platinum) and time period (yearly or monthly) and the 
function should automatically take these inputs while identifying the top 10 customers.

In [ ]:
import pandas as pd

def top_customers_by_city(merged_data, product, time_period):
    """
    Find top 1